In [11]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list((rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list((rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-(mockForcastDay+forecastDay)] & np.log
    ds = rawArrayDatas[0][:-(mockForcastDay+forecastDay)]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-(mockForcastDay+forecastDay)]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of (mockForcastDay+forecastDay)  rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]
    testY= rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
#         listedLogPredict=test_predict[-1].tolist()
#     return [np.exp(y) for y in listedLogPredict]
    return test_predict[-1].tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm


In [12]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [14]:
columns=['ds','y']
df=pd.read_table('webMonth36.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.2)
print(len(df), forecastDay)


36 7


In [15]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
rawArrayDatas

[['2013-02-01',
  '2013-03-01',
  '2013-04-01',
  '2013-05-01',
  '2013-06-01',
  '2013-07-01',
  '2013-08-01',
  '2013-09-01',
  '2013-10-01',
  '2013-11-01',
  '2013-12-01',
  '2014-01-01',
  '2014-02-01',
  '2014-03-01',
  '2014-04-01',
  '2014-05-01',
  '2014-06-01',
  '2014-07-01',
  '2014-08-01',
  '2014-09-01',
  '2014-10-01',
  '2014-11-01',
  '2014-12-01',
  '2015-01-01',
  '2015-02-01',
  '2015-03-01',
  '2015-04-01',
  '2015-05-01',
  '2015-06-01',
  '2015-07-01',
  '2015-08-01',
  '2015-09-01',
  '2015-10-01',
  '2015-11-01',
  '2015-12-01',
  '2016-01-01'],
 [4432.5,
  2776.5161290000001,
  2286.6333329999998,
  1913.451613,
  1650.9333329999999,
  2040.0333329999999,
  2734.0645159999999,
  13740.266669999999,
  10564.35484,
  7608.3999999999996,
  8706.1935480000011,
  26366.56667,
  24504.89286,
  2169.0322579999997,
  1879.0,
  2449.5483870000003,
  1487.9000000000001,
  1903.580645,
  3121.333333,
  8773.6333329999998,
  9429.2903230000011,
  4370.5666670000001,
  412

In [17]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'month') #0은 processId

[step: 0] loss: 5.611518859863281
[step: 1] loss: 5.297228813171387
[step: 2] loss: 5.009056568145752
[step: 3] loss: 4.743826866149902
[step: 4] loss: 4.499009609222412
[step: 5] loss: 4.273017406463623
[step: 6] loss: 4.065521240234375
[step: 7] loss: 3.8776276111602783
[step: 8] loss: 3.712122917175293
[step: 9] loss: 3.5735976696014404
[step: 10] loss: 3.4679834842681885
[step: 11] loss: 3.4005162715911865
[step: 12] loss: 3.3710978031158447
[step: 13] loss: 3.3691771030426025
[step: 14] loss: 3.375135898590088
[step: 15] loss: 3.3701696395874023
[step: 16] loss: 3.345579147338867
[step: 17] loss: 3.3039486408233643
[step: 18] loss: 3.25403094291687
[step: 19] loss: 3.2048285007476807
[step: 20] loss: 3.162043809890747
[step: 21] loss: 3.127261161804199
[step: 22] loss: 3.0989184379577637
[step: 23] loss: 3.0739097595214844
[step: 24] loss: 3.048980712890625
[step: 25] loss: 3.0215423107147217
[step: 26] loss: 2.9899909496307373
[step: 27] loss: 2.953716516494751
[step: 28] loss: 2

[step: 270] loss: 0.04434351250529289
[step: 271] loss: 0.03949549421668053
[step: 272] loss: 0.036308176815509796
[step: 273] loss: 0.03470148146152496
[step: 274] loss: 0.034802988171577454
[step: 275] loss: 0.03599502146244049
[step: 276] loss: 0.03722159191966057
[step: 277] loss: 0.0388123095035553
[step: 278] loss: 0.038601115345954895
[step: 279] loss: 0.0388236828148365
[step: 280] loss: 0.03652128949761391
[step: 281] loss: 0.03472968935966492
[step: 282] loss: 0.032148417085409164
[step: 283] loss: 0.030379198491573334
[step: 284] loss: 0.0293203666806221
[step: 285] loss: 0.02897919900715351
[step: 286] loss: 0.02913524955511093
[step: 287] loss: 0.029504068195819855
[step: 288] loss: 0.03024415299296379
[step: 289] loss: 0.030851095914840698
[step: 290] loss: 0.03240402042865753
[step: 291] loss: 0.03355005010962486
[step: 292] loss: 0.036635030061006546
[step: 293] loss: 0.0380077101290226
[step: 294] loss: 0.04149849712848663
[step: 295] loss: 0.039483971893787384
[step: 

[step: 542] loss: 0.000734774221200496
[step: 543] loss: 0.0008249462116509676
[step: 544] loss: 0.0010573087492957711
[step: 545] loss: 0.0011574135860428214
[step: 546] loss: 0.001027655671350658
[step: 547] loss: 0.0007737537962384522
[step: 548] loss: 0.0006082087638787925
[step: 549] loss: 0.0006244428222998977
[step: 550] loss: 0.0007500886567868292
[step: 551] loss: 0.0008436397765763104
[step: 552] loss: 0.0008156262338161469
[step: 553] loss: 0.0006992415874265134
[step: 554] loss: 0.0005841598613187671
[step: 555] loss: 0.0005408097058534622
[step: 556] loss: 0.0005685418727807701
[step: 557] loss: 0.0006140595069155097
[step: 558] loss: 0.0006271435413509607
[step: 559] loss: 0.0005907340673729777
[step: 560] loss: 0.0005335572641342878
[step: 561] loss: 0.000490402162540704
[step: 562] loss: 0.0004802475159522146
[step: 563] loss: 0.0004960047663189471
[step: 564] loss: 0.0005133957020007074
[step: 565] loss: 0.0005135115934535861
[step: 566] loss: 0.000489737547468394
[ste

[step: 813] loss: 0.0020715901628136635
[step: 814] loss: 0.0002191196836065501
[step: 815] loss: 0.0029675415717065334
[step: 816] loss: 0.004522689152508974
[step: 817] loss: 0.0027444795705378056
[step: 818] loss: 0.0002596623089630157
[step: 819] loss: 0.0010403041960671544
[step: 820] loss: 0.002941056387498975
[step: 821] loss: 0.002185933291912079
[step: 822] loss: 0.00039703852962702513
[step: 823] loss: 0.00033902956056408584
[step: 824] loss: 0.001572462497279048
[step: 825] loss: 0.001726931077428162
[step: 826] loss: 0.000510465179104358
[step: 827] loss: 0.00012924101611133665
[step: 828] loss: 0.0009065982885658741
[step: 829] loss: 0.001165009569376707
[step: 830] loss: 0.0005140042048878968
[step: 831] loss: 4.8918009269982576e-05
[step: 832] loss: 0.0004336648853495717
[step: 833] loss: 0.0008066933369264007
[step: 834] loss: 0.0004554681945592165
[step: 835] loss: 5.9509540733415633e-05
[step: 836] loss: 0.00021230147103779018
[step: 837] loss: 0.0005040602991357446
[

[step: 57] loss: 2.4962024688720703
[step: 58] loss: 2.4505655765533447
[step: 59] loss: 2.407627820968628
[step: 60] loss: 2.3695504665374756
[step: 61] loss: 2.336663007736206
[step: 62] loss: 2.310565948486328
[step: 63] loss: 2.2900216579437256
[step: 64] loss: 2.270960569381714
[step: 65] loss: 2.2511446475982666
[step: 66] loss: 2.2301721572875977
[step: 67] loss: 2.206718683242798
[step: 68] loss: 2.179894208908081
[step: 69] loss: 2.1514296531677246
[step: 70] loss: 2.123506546020508
[step: 71] loss: 2.096935749053955
[step: 72] loss: 2.0735206604003906
[step: 73] loss: 2.0545854568481445
[step: 74] loss: 2.037886619567871
[step: 75] loss: 2.021355628967285
[step: 76] loss: 2.0032732486724854
[step: 77] loss: 1.9824376106262207
[step: 78] loss: 1.9609599113464355
[step: 79] loss: 1.9401494264602661
[step: 80] loss: 1.9211300611495972
[step: 81] loss: 1.9042385816574097
[step: 82] loss: 1.8878525495529175
[step: 83] loss: 1.871283769607544
[step: 84] loss: 1.8536057472229004
[st

[step: 287] loss: 0.22450785338878632
[step: 288] loss: 0.2225727140903473
[step: 289] loss: 0.2211959958076477
[step: 290] loss: 0.21882732212543488
[step: 291] loss: 0.21783587336540222
[step: 292] loss: 0.2154659479856491
[step: 293] loss: 0.21432755887508392
[step: 294] loss: 0.21244584023952484
[step: 295] loss: 0.21084913611412048
[step: 296] loss: 0.20954985916614532
[step: 297] loss: 0.20764116942882538
[step: 298] loss: 0.20654666423797607
[step: 299] loss: 0.20480193197727203
[step: 300] loss: 0.20343637466430664
[step: 301] loss: 0.20213951170444489
[step: 302] loss: 0.20050033926963806
[step: 303] loss: 0.19937115907669067
[step: 304] loss: 0.19788701832294464
[step: 305] loss: 0.19653785228729248
[step: 306] loss: 0.19537323713302612
[step: 307] loss: 0.19391606748104095
[step: 308] loss: 0.19274236261844635
[step: 309] loss: 0.19151706993579865
[step: 310] loss: 0.19016942381858826
[step: 311] loss: 0.18906110525131226
[step: 312] loss: 0.18783175945281982
[step: 313] los

[step: 509] loss: 0.07498130947351456
[step: 510] loss: 0.07464370876550674
[step: 511] loss: 0.07430645823478699
[step: 512] loss: 0.07396921515464783
[step: 513] loss: 0.07363186776638031
[step: 514] loss: 0.07329446822404861
[step: 515] loss: 0.07295721024274826
[step: 516] loss: 0.07261984795331955
[step: 517] loss: 0.0722821056842804
[step: 518] loss: 0.07194439321756363
[step: 519] loss: 0.07160631567239761
[step: 520] loss: 0.07126791775226593
[step: 521] loss: 0.07092909514904022
[step: 522] loss: 0.07058980315923691
[step: 523] loss: 0.07024998217821121
[step: 524] loss: 0.06990958750247955
[step: 525] loss: 0.06956847012042999
[step: 526] loss: 0.06922657787799835
[step: 527] loss: 0.06888367235660553
[step: 528] loss: 0.06853986531496048
[step: 529] loss: 0.06819502264261246
[step: 530] loss: 0.06784899532794952
[step: 531] loss: 0.06750169396400452
[step: 532] loss: 0.06715299934148788
[step: 533] loss: 0.06680278480052948
[step: 534] loss: 0.06645100563764572
[step: 535] l

[step: 728] loss: 0.006191725376993418
[step: 729] loss: 0.006136439274996519
[step: 730] loss: 0.006097504869103432
[step: 731] loss: 0.006068509072065353
[step: 732] loss: 0.006043832749128342
[step: 733] loss: 0.006017602514475584
[step: 734] loss: 0.005988866090774536
[step: 735] loss: 0.005953904706984758
[step: 736] loss: 0.005916525609791279
[step: 737] loss: 0.005873286165297031
[step: 738] loss: 0.005830179899930954
[step: 739] loss: 0.00578340794891119
[step: 740] loss: 0.00573942344635725
[step: 741] loss: 0.005694231484085321
[step: 742] loss: 0.005653929430991411
[step: 743] loss: 0.005614497698843479
[step: 744] loss: 0.005582223646342754
[step: 745] loss: 0.005552995018661022
[step: 746] loss: 0.005534997675567865
[step: 747] loss: 0.005523262079805136
[step: 748] loss: 0.005530651658773422
[step: 749] loss: 0.0055505214259028435
[step: 750] loss: 0.005606491584330797
[step: 751] loss: 0.005688084755092859
[step: 752] loss: 0.00584407290443778
[step: 753] loss: 0.0060520

[step: 956] loss: 0.0016078801127150655
[step: 957] loss: 0.0015977324219420552
[step: 958] loss: 0.0015871268697082996
[step: 959] loss: 0.0015764321433380246
[step: 960] loss: 0.0015660362550988793
[step: 961] loss: 0.0015560663305222988
[step: 962] loss: 0.0015464633470401168
[step: 963] loss: 0.0015370827168226242
[step: 964] loss: 0.0015277510974556208
[step: 965] loss: 0.0015183056239038706
[step: 966] loss: 0.0015087034553289413
[step: 967] loss: 0.0014989293413236737
[step: 968] loss: 0.0014891078462824225
[step: 969] loss: 0.0014793297741562128
[step: 970] loss: 0.0014696929138153791
[step: 971] loss: 0.0014602026203647256
[step: 972] loss: 0.0014508459717035294
[step: 973] loss: 0.001441590371541679
[step: 974] loss: 0.0014323971699923277
[step: 975] loss: 0.0014232475077733397
[step: 976] loss: 0.001414117286913097
[step: 977] loss: 0.0014049519086256623
[step: 978] loss: 0.0013957963092252612
[step: 979] loss: 0.001386631978675723
[step: 980] loss: 0.0013774940744042397
[st

In [18]:
len(answer)

7

In [19]:
answer

[10258.6474609375,
 12691.853515625,
 -739.6880493164062,
 -589.4653930664062,
 5872.23388671875,
 5039.17236328125,
 -4128.912109375]

In [20]:
len(rawArrayDatas[1])

36